In [7]:
import os  

def file_name(file_dir,suffix=None):   
    L=[]   
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if suffix !=None and os.path.splitext(file)[1] != suffix:
                continue
            L.append(os.path.join(dirpath, file))  
    return L 

In [15]:
file_name('checkpoint','.t7')

['checkpoint\\classification_ckpt.t7',
 'checkpoint\\contrastive_ckpt.t7',
 'checkpoint\\Fisher_ckpt.t7',
 'checkpoint\\triplet_ckpt.t7',
 'checkpoint\\triplet_HardestNegativeTripletSelector_ckpt.t7']

In [38]:
import os
import torch
import torch.backends.cudnn as cudnn
from torchvision_qinxiao import datasets
import torchvision.transforms as transforms
from models.lenet import LeNet
import numpy as np
from utils_qinxiao import KNNEvaluation,progress_bar
from evaluations.NMI import NMI
from evaluations.recall_at_k import Recall_at_ks
from scipy.spatial import distance_matrix
def evaluate(embeddings,targets):
#     knn_score=KNNEvaluation(embeddings,targets)
#     return knn_score
#     NMI_score=NMI(embeddings,targets,n_cluster=5)
#     return NMI_score
    dist_matrix=distance_matrix(embeddings,embeddings)
    Recall_k_score=Recall_at_ks(sim_mat=dist_matrix, data='product', gallery_ids=int(5e2)*list(range(10)))
    return Recall_k_score


if __name__ == '__main__':
    # Data
    print('==> Preparing data..')
    bs=1024
    kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../Embedding_data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=bs, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../Embedding_data', train=False, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=bs, shuffle=True, **kwargs)

    # Model
    print('==> Building model..')
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net = LeNet(5)
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    fileList=file_name('checkpoint','.t7')
    for ckpt_file in fileList:
        print('==> Resuming from checkpoint '+ckpt_file)
        assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
        checkpoint = torch.load(ckpt_file)
        net.load_state_dict(checkpoint['net'])
        best_acc = checkpoint['acc']
        net.eval()
        with torch.no_grad():
            all_embeddings=np.zeros((len(test_loader.dataset),84))
            all_targets=np.zeros(len(test_loader.dataset))
            for batch_idx, (inputs, targets) in enumerate(test_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                _,embeddings = net(inputs)
                all_embeddings[batch_idx*bs:batch_idx*bs+embeddings.shape[0],:]=embeddings.cpu().numpy()
                all_targets[batch_idx*bs:batch_idx*bs+targets.shape[0]]=targets.cpu().numpy()
                progress_bar(batch_idx, len(test_loader))
        e1=evaluate(all_embeddings,all_targets)
        print(e1)
        

==> Preparing data..
==> Building model..
==> Resuming from checkpoint checkpoint\Binomial_ckpt.t7
 [=================================>......]  Step: 7ms | Tot: 503ms                                                    6/6 


IndexError: boolean index did not match indexed array along dimension 0; dimension is 5139 but corresponding boolean dimension is 5000